# Let's practice !
In this exercice, you will implement a vertical 2D jumper that must reach a height of 1 meter with one torque-actuated leg made of two segments.
The vertical degree of freedom actuation must be forced to zero, to mimic free flying conditions.
The lower segment of the leg must be constrained with a contact point in the first phase which will then be released in the second phase.
The main objective is to minimize torques during each phase.
The first phase will last for 0.2s with 40 nodes and the second will last for 0.4s with 40 nodes.

The motion will look like this :

![SegmentLocal](img/slider.gif)

In [ ]:
import biorbd
from bioptim import *

In [ ]:
model = biorbd.Model("models/Slider1Leg.bioMod"), biorbd.Model("models/Slider1Leg.bioMod")
nq = model[0].nbQ()
ntau = model[0].nbGeneralizedTorque()

# This is your turn
n_shooting = 40, 40
final_time = 0.2, 0.4
tau_min, tau_max, tau_init = -200, 200, 0

dynamics = DynamicsList()
dynamics.add(DynamicsFcn.TORQUE_DRIVEN_WITH_CONTACT)
dynamics.add(DynamicsFcn.TORQUE_DRIVEN)

objective_functions = ObjectiveList()
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_TORQUE, weight=0.1, phase=0)
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_TORQUE, weight=0.1, phase=1)

x_bounds = BoundsList()
x_bounds.add(bounds=QAndQDotBounds(model[0]))
x_bounds.add(bounds=QAndQDotBounds(model[1]))

x_bounds[0][:3, 0] = [0, 3*np.pi / 8, -3*np.pi / 4]
x_bounds[0][3:, 0] = 0
x_bounds[0].min[0, -1] = 0
x_bounds[0].max[0, -1] = 0.25 

x_bounds[1][0, -1] = 1
x_bounds[1][3, -1] = 0

u_bounds = BoundsList()
u_bounds.add([tau_min] * nq,
             [tau_max] * ntau)
u_bounds.add([tau_min] * ntau,
             [tau_max] * ntau)

u_bounds[0][0, :] = 0
u_bounds[1][0, :] = 0

x_init = InitialGuessList()
x_init.add([0, 3*np.pi / 8, -3*np.pi / 4] + [0] * nq)  
x_init.add([0, 0, 0] + [0] * nq)  

u_init = InitialGuessList()
u_init.add([tau_init] * ntau)
u_init.add([tau_init] * ntau)

# Constraints
constraints = ConstraintList()
constraints.add(
    ConstraintFcn.CONTACT_FORCE,
    min_bound=0,
    max_bound=np.inf,
    node=Node.ALL,
    contact_force_idx=0,  # z axis
)

ocp =  OptimalControlProgram(
    model,
    dynamics,
    n_shooting,
    final_time,
    x_init,
    u_init,
    x_bounds,
    u_bounds,
    objective_functions=objective_functions,
    constraints=constraints,
    n_threads=8,
)



In [ ]:
sol = ocp.solve(show_online_optim=False)


In [ ]:
sol.graphs(automatically_organize=False)